# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
data = run_engine(data)

100%|██████████████████████████████| 3521001/3521001 [09:55<00:00, 5910.27it/s]

pl    0.28463
dtype: float64
--------------------------------------
              DateTime_frmt          close_type       pl
3506902 2021-02-26 19:29:13         simple_stop -0.00040
3507305 2021-02-26 19:36:54         simple_stop -0.00045
3508094 2021-02-26 19:50:39  simple_take_profit  0.00018
3509001 2021-02-26 19:59:04         simple_stop -0.00063
3509861 2021-02-26 20:06:39         simple_stop -0.00060
3510001 2021-02-26 20:09:24  simple_take_profit  0.00019
3512603 2021-02-26 20:49:16  simple_take_profit  0.00018
3513501 2021-02-26 20:59:13         simple_stop -0.00081
3514434 2021-02-26 21:06:09  simple_take_profit  0.00044
3515116 2021-02-26 21:14:20  simple_take_profit  0.00033
3517389 2021-02-28 22:14:46         simple_stop -0.00104
3519067 2021-02-28 23:15:13  simple_take_profit  0.00029
3519257 2021-02-28 23:17:15  simple_take_profit  0.00029
3520621 2021-02-28 23:42:04         simple_stop -0.00050
3520892 2021-02-28 23:48:18  simple_take_profit  0.00025


In [3]:
generate_result_report(data)

--------------------------------------
Run Complete : data/ema_df-(2021-2021)-(1-2)-(1-28).csv
--------------------------------------


--------------------------------------
Net PL
0.2846
--------------------------------------


--------------------------------------
Monthly PL
                pl
month_val         
1          0.16854
2          0.11609
--------------------------------------


--------------------------------------
Count of transactions by Close Type
..................
                      pl
close_type              
simple_stop         1164
simple_take_profit  3444
--------------------------------------


--------------------------------------
Mean value of transactions by Close Type
..................
                          pl
close_type                  
simple_stop        -0.000566
simple_take_profit  0.000274
--------------------------------------


--------------------------------------
Sum by Close Type
..................
                         pl
close_ty

In [4]:
plot_graph(data)